In [1]:
import numpy as np 
from matplotlib import pyplot as plt

plt.rcdefaults()
plt.style.use("seaborn-whitegrid")
plt.rc("figure", figsize=(11.2, 6.3))
plt.rc("font", size=12)
plt.rc("axes", edgecolor="white")
plt.rc("legend", frameon=True, framealpha=0.8, facecolor="white", edgecolor="white")

# 1 Numerical Linear Algebra I

## Basics
### Problem Statement
For $A=(a_{ij})\in\mathbb{K}^{n\times n}$ and $\ b=(b_i)\in\mathbb{K}^n$ find or determine the existence of $x=(x_i)\in\mathbb{K}^n$ such that $Ax=b,$ where $\mathbb{K}=\mathbb{R}$ or $\mathbb{C}.$
### Results from Linear Algebra
$Ax=b$ has exactly one solution if and only if A is regular, which is equivalent to $\det A \neq 0$ and to $\textrm{rank} A = n.$ In this case $x=A^{-1}b.$

Furthermore using Cramer's rule $x$ can be determined by calculating $x_i = \frac{\det A^{(i)}}{\det A} = \frac{1}{\det A} \textrm{adj}(A)b.$ $A^{(i)}$ is given by replacing the $i$-th column of $A$ with $b$ and $\textrm{adj}(A)_{jk} = (-1)^{j+k} S_{kj}(A)$ is the adjoint matrix of $A,$ where $S_{kj}(A) \in\mathbb{K}^{(n-1)\times (n-1)}$ is given by deleting the $k$-th row and $j$-th column of $A.$

However the complexity of calculating the determinant is very high, namely $O(n!),$ and therefore using Cramer's rule is not feasible.

## Direct Methods
### Gauß Algorithm
Solving a system with a lower or upper triangular matrix is straightforward using forward or back substitution respectively and a solution exists if no diagonal element is zero.

In [2]:
def solveLower(L, b):
    L = np.array(L, dtype = np.double)
    n = L.shape[0]
    x = np.zeros(n, dtype = np.double)
    
    for i in range(n):
        if L[i,i] == 0:
            raise Exception("Matrix is singular.")
        else:
            x[i] = (b[i] - sum(L[i,:i] * x[:i])) / L[i,i]
    
    return x

In [3]:
def solveUpper(U, b):
    return solveLower(U[::-1,::-1], b[::-1])[::-1] #solves flipped to lower triangular system

For an $n\times n$ triangular matrix there are $k-1$ multiplications, $k-1$ additions and 1 division in the $k$-th step. Thus for $n$ steps there are $\frac{n(n-1)}{2}$ multiplications, $\frac{n(n-1)}{2}$ additions and $n$ divisions. Since these operations are roughly equivalent in terms of complexity on a modern computer there are $n^2$ operations and therefore the complexity is $O(n^2).$

The Gauß algorithm first decomposes a given matrix into an upper and lower triangular matrix.

In [4]:
def LUdecomp(A):
    U = np.array(A, dtype = np.double)
    n = U.shape[0]
    L = np.eye(n, dtype = np.double)
    
    #loops over rows
    for i in range(n-1):
        
        #updates L and eliminates entries below i in U
        L[i+1:,i] = U[i+1:,i] / U[i,i]
        U[i+1:] = U[i+1:] - L[i+1:,i][:, np.newaxis] * U[i]
    
    return L, U

In [5]:
A = [[2,1,1,0],[4,3,3,1],[8,7,9,5],[6,7,9,8]]
L, U = LUdecomp(A)
print(L@U - A)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


There are $2(n-k+1)$ operations in the inner loop of the LU decomposition which results in a complexity of $O(n^3).$ This naive approach is not suitable for all regular matrices since it fails if there is a zero on the diagonal of the given matrix, e.g. $\big(\begin{smallmatrix} 0 & 1 \\ 1 & 0 \end{smallmatrix}\big).$ Because of floating point arithmetic errors a matrix with a diagonal element very close to zero, i.e. smaller than the machine precision $\textrm{eps},$ may not get properly decomposed by the LU decomposition:

In [6]:
A = [[10**(-20),1],[1,1]]
L, U = LUdecomp(A)
print(L@U - A)

[[ 0.  0.]
 [ 0. -1.]]


The naive Gauß algorithm now uses the LU decomposition to first solve $Ax=LUx=Ly=b$ for $y$ and then $Ux=y$ for the desired $x.$

In [7]:
def GaußAlgorithm(A, b):
    L, U = LUdecomp(A)
    y = solveLower(L, b)
    return solveUpper(U, y)

Since the higher order complexity comes from the LU decomposition the Gauß algorithm also has complexity $O(n^3).$

In [8]:
A = [[2,1,1,0],[4,3,3,1],[8,7,9,5],[6,7,9,8]]
b = [1,3,4,7]
x = GaußAlgorithm(A, b)
print(A@x - b)

[0. 0. 0. 0.]


### Pivoting
By permuting rows of $A$ we can get the instability under control. Observe that in the $k$-th step of the Gauß algorithm multiples of the $k$-th row are subtracted from the rows $k+1,...,n$ of $A$ to get zeros in the $k$-th entry of that row. In this step the product of an entry of the $k$-th column and an element of the $k$-th row is subtracted and is then divided by the so called pivot element $A_{kk}.$ There is specific reason for using the $k$-th row and column in this step. Using row $i$ and column $j$ for $k < i,j \le n$ results in the pivot element $A_{ij}.$ Choosing such a pivot element which is not equal to zero is called pivoting.

#### Column Pivoting
First we will consider pivoting just the columns since otherwise there would be $(n-k)^2$ options in the $k$-th step resulting in a complexity of $O(n^3)$ for choosing the pivot element. Choosing the element with the maximal absolute value from the lower part of the $k$-th column and swapping the rows results in small entries in $L$ which suppresses rounding errors.

This algorithm returns lower and upper triangle matrices $L$ and $U$ as well as a permutation matrix $P$ such that $LU = PA.$

In [9]:
def LUPdecomp(A):
    U = np.array(A, dtype = np.double)
    n = U.shape[0]
    L = np.eye(n, dtype = np.double)
    P = np.eye(n, dtype = np.double)
    
    #loops over rows
    for i in range(n-1):
        
        #chooses pivot column index
        pivot = i + np.abs(U[i:,i]).argmax()
        
        #swaps rows
        U[[i,pivot]] = U[[pivot,i]]
        L[[i,pivot],:i] = L[[pivot,i],:i]
        P[[i,pivot]] = P[[pivot,i]]
        
        #updates L and eliminates entries below i in U
        L[i+1:,i] = U[i+1:,i] / U[i,i]
        U[i+1:] = U[i+1:] - L[i+1:,i][:, np.newaxis] * U[i]
        
    return L, U, P

Since only $O(n^2)$ comparisons are needed to find the pivot elements the complexity is the same as the naive version with $O(n^3).$ Additionally a permutation matrix has to be stored which can be done efficiently as a vector $p$ of $n$ integers with $p_i = \sigma(i).$

In [10]:
A = [[2,1,1,0],[4,3,3,1],[8,7,9,5],[6,7,9,8]]
L, U, P = LUPdecomp(A)
print(L@U - P@A)

[[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 2.22044605e-16]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]]


Using the LUP decomposition to solve the system $Ax=b$ we can rewrite it to $PAx=Pb \iff LUx=Pb.$ The Gauß algorithm with column pivoting first solves $Ly=Pb$ and then $Ux=y.$ Again the complexity is $O(n^3).$

In [11]:
def GaußAlgorithmColumnPivot(A, b):
    L, U, P = LUPdecomp(A)
    y = solveLower(L, P@b)
    return solveUpper(U, y)

In [12]:
A = [[2,1,1,0],[4,3,3,1],[8,7,9,5],[6,7,9,8]]
b = [1,3,4,7]
x = GaußAlgorithmColumnPivot(A, b)
print(A@x - b)

[0.00000000e+00 1.33226763e-15 1.77635684e-15 0.00000000e+00]


#### Full Pivoting
As stated before the complexity of finding the full pivot element is of order $O(n^3)$ and therefore in practice seldom used. Nonetheless for the sake of completeness the following algorithm decomposes a given square matrix $A$ into lower and upper triangle matrices, $L$ and $U$, and two permutations matrices, $P$ and $Q$, such that $PAQ = LU.$

In [13]:
def LUPQdecomp(A):
    U = np.array(A, dtype = np.double)
    n = U.shape[0]
    L = np.eye(n, dtype = np.double)
    P = np.eye(n, dtype = np.double)
    Q = np.eye(n, dtype = np.double)
    
    #loops over rows
    for i in range(n-1):
        
        #chooses pivot index
        pivotRow, pivotCol = np.unravel_index(U[i:,i:].argmax()+(2*n-i)*i, (n,n))
        
        #swaps rows
        U[[i,pivotRow]] = U[[pivotRow,i]]
        L[[i,pivotRow],:i] = L[[pivotRow,i],:i]
        P[[i,pivotRow]] = P[[pivotRow,i]]
        
        #swaps columns
        U[:,[i,pivotCol]] = U[:,[pivotCol,i]]
        Q[:,[i,pivotCol]] = Q[:,[pivotCol,i]]
        
        #updates L and eliminates entries below i in U
        L[i+1:,i] = U[i+1:,i] / U[i,i]
        U[i+1:] = U[i+1:] - L[i+1:,i][:, np.newaxis] * U[i]
    
    return L, U, P, Q

In [14]:
A = [[2,1,1,0],[4,3,3,1],[8,7,9,5],[6,7,9,8]]
L, U, P, Q = LUPQdecomp(A)
print(P@A@Q - L@U)

[[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 4.44089210e-16 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 1.11022302e-16]]


### Correction Iteration
If a higher precision arithmetic than was used for the Gauß algorithm is available one may use a correction iteration to get a better approximation of the real solution. If the precision is the same then the new approximation is not better than the previous.

In [15]:
def correction(A, x, b, L, U, P):
    r = b - A@x
    y = solveLower(L, P@r)
    z = solveUpper(U, y)
    return x + z

## Error Analysis
### Regularity Measures
To estimate the error of our approximations it is useful to know _how_ regular a matrix is.
#### Determinant
The determinant only determines if a given matrix is regular or not: $\det A \ne 0 \iff A\ \textrm{regular}.$ A multiple of a matrix is not more or less regular but since $\det(\lambda A) = \lambda^n \det(A)$ the determinant is not useful in this context.
#### Condition Number
For a given matrix $A$ and matrix norm $|\!|\cdot |\!|_M$ we define the _condition number_ $\kappa_M(A):=|\!|A^{-1}|\!|_M\,|\!|A|\!|_M.$ If $A$ is singular then $\kappa_M(A):=\infty.$

The condition number satisfies $\kappa(\lambda A) = \kappa(A)$. The definition for singular matrices is justified since for a singular matrix $B$ with $\textrm{rank}(B)=n-1$ a sequence $(A_k)_k$ of regular matrices that converges to $B$ satisfies $\kappa(A_k)\rightarrow\infty.$ Furthermore $\kappa(A)\ge |\!|AA^{-1}|\!|=|\!|I|\!|=1.$ Therefore a very regular matrix has a condition number close to 1 whereas a nearly singular matrix has a very large one.

For the system $Ax=b$ and an approximation $\tilde{x}$ of $x$ it holds that $\dfrac{|\!|x - \tilde{x}|\!|}{|\!|x|\!|} \le \kappa(A)\ \dfrac{|\!|b-A\tilde{x}|\!|}{|\!|b|\!|}.$

For all regular diagonal matrices $\kappa_\infty(DA)$ is minimal for $D=\textrm{diag}\big((\sum_{j=1}^n |A_{1j}|)^{-1},\dots,(\sum_{j=1}^n |A_{nj}|)^{-1}\big)$ thus if $\sum_{j=1}^n |(DA)_{ij}|=1$ for $i=1,\dots,n.$ So to get a meaningful measure one should first appropriatly scale $A$ which is illustrated by the following example.

Let $A=\big(\begin{smallmatrix} 300 & 1 \\ 1 & 1 \end{smallmatrix}\big).$ Then $A^{-1}=\frac{1}{299}\big(\begin{smallmatrix} 1 & -1 \\ -1 & 300 \end{smallmatrix}\big)$ and $\kappa_\infty(A)=\frac{90601}{299}\approx 300.$ If we scale $A$ by $D=\textrm{diag}(\frac{1}{301},\frac{1}{2})$ then $\kappa(DA)=\frac{901}{303}\approx 3,$ which is a decrease by a factor of $100$.

A matrix $A$ for which this is already the case, i.e. for $\sum_{j=1}^n |A_{ij}|=1\ \forall i\in \{1,\dots,n\},$ is called _equilibrated_. 

To compute $\kappa(A)$ one has to know the inverse, which is harder to compute than a solution of a linear system. In order to still get an estimate we can use the following:

In [16]:
def conditionEstimate(A):
    n = len(A)
    L,U,P = LUPdecomp(np.array(A).T.conj()) #decomposes the conjugate transpose A* of A
    e = np.ones(n)
    v = solveUpper(U, solveLower(L, e)) #solves A* v = e
    d = np.sign(v)
    w = GaußAlgorithmColumnPivot(A, d) #solves Aw = d
    j = w.argmax()
    ej = np.zeros(n)
    ej[j] = 1
    f = solveUpper(U, solveLower(L, P@e)) #solves A* f = e
    s = np.abs(f)@e #computes inner produc <|f|,e>
    norm = np.max(np.abs(A).sum(axis=0)) #computes ||A||_oo
    return s * norm

If the LU decomposition is known beforehand then the complexity of the estimation is $O(n^2)$ otherwise $O(n^3).$

In [17]:
A = [[300,1],[1,1]]
print("estimated condition number:", conditionEstimate(A))
print("actual condition number:   ", 90601/299)

estimated condition number: 301.0
actual condition number:    303.0133779264214


#### Singularity Distance
The _singularity distance_ of a matrix $A$ is defined as $\textrm{sing}(A):=\inf\{\delta \ge 0\ :\ \exists B$ singular with $|B-A|\le\delta |A|\}.$ This means that for a neighborhood of $A$ with a componentwise error of order $\textrm{sing}(A)$ there exists a singular matrix but not for a smaller error.

Properties of $\textrm{sing}(A)$:
- $0 \le \textrm{sing}(A) \le 1,$
- $\textrm{sing}(A) = 0 \iff A$ singular,
- $\textrm{sing}(D_1AD_2) = \textrm{sing}(A)$ for any regular diagonal matrices $D_1$ and $D_2.$ Thus the singularity distance is independent of scaling.
Unfortunately this distance is not very practical to compute. But if $A$ is regular then $\textrm{sing}(A) \ge \frac{1}{\kappa_\infty(A)}$

### Perturbations
The matrix $A$ might not be known exactly. The following result provides information about the effects of the inaccuracies.

Let $A\in\mathbb{K}^{n\times n}, b,\tilde{x}\in\mathbb{K}^n$ and $\Delta A\in\mathbb{R}^{n\times n}, \Delta b\in\mathbb{R}^n$ arbitrary and non-negative. Then the following are equivalent:
- $\exists \tilde{A}\in\mathbb{K}^{n\times n}, \tilde{b}\in\mathbb{K}^n$ such that $\tilde{A}\tilde{x}=\tilde{b}, |\tilde{A}-A|\le\Delta A$ and $|\tilde{b}-b|\le\Delta b.$
- $|b-A\tilde{x}| \le \Delta b + \Delta A |\tilde{x}|$

If for example $A$ and $b$ have the same relative error $\epsilon$, i.e. $\Delta A = \epsilon |A|, \Delta b = \epsilon |b|$, then $\tilde{x}$ solves a nearby system $\tilde{A}\tilde{x}=\tilde{b}$ with $|\tilde{A}-A|\le\Delta A$ and $|\tilde{b}-b|\le\Delta b$ if $|b-A\tilde{x}| \le \epsilon (|b|+|A||\tilde{x}|).$ This provides the smallest $\epsilon$ such that the solution is acceptable by a given standard.

### Gauß Algorithm Approximation Error
While column pivoting has addressed some problems with the naive approach, some still remain. For example the following matrix was not decomposed correctly without pivoting, see Gauß algorithm above, but correctly with column pivoting:

In [18]:
A = [[1e-20,1],[1,1]]
L,U,P = LUPdecomp(A)
print(L@U-P@A)

[[0. 0.]
 [0. 0.]]


On the other hand if this matrix is scaled by $D=\textrm{diag}(10^{20},1)$ then column pivoting fails.

In [19]:
DA = [[1,10**20],[1,1]]
L,U,P = LUPdecomp(DA)
print(L@U-P@DA)

[[0.0 0.0]
 [0.0 -1.0]]


Also the following matrix $A$ has 

In [20]:
n = 6
A = -np.tri(n) + np.diag(2*np.ones(n))
A.T[-1] = np.ones(n)
L,U,P = LUPdecomp(A)
print("A =", A, "\nL =", L, "\nU =", U, sep="\n")

A =
[[ 1.  0.  0.  0.  0.  1.]
 [-1.  1.  0.  0.  0.  1.]
 [-1. -1.  1.  0.  0.  1.]
 [-1. -1. -1.  1.  0.  1.]
 [-1. -1. -1. -1.  1.  1.]
 [-1. -1. -1. -1. -1.  1.]]

L =
[[ 1.  0.  0.  0.  0.  0.]
 [-1.  1.  0.  0.  0.  0.]
 [-1. -1.  1.  0.  0.  0.]
 [-1. -1. -1.  1.  0.  0.]
 [-1. -1. -1. -1.  1.  0.]
 [-1. -1. -1. -1. -1.  1.]]

U =
[[ 1.  0.  0.  0.  0.  1.]
 [ 0.  1.  0.  0.  0.  2.]
 [ 0.  0.  1.  0.  0.  4.]
 [ 0.  0.  0.  1.  0.  8.]
 [ 0.  0.  0.  0.  1. 16.]
 [ 0.  0.  0.  0.  0. 32.]]


The first problem can be fixed by making sure that $A$ is equilibrated before column pivoting.

In [21]:
def modLUPdecomp(A):
    U = np.array(A, dtype = np.double)
    n = U.shape[0]
    L = np.eye(n, dtype = np.double)
    P = np.eye(n, dtype = np.double)
    
    #loops over rows
    for i in range(n-1):
        
        #chooses pivot column index from equilibrated matrix
        pivot = i + (np.abs(U[i:,i]) / np.abs(U[i:,:]).sum(axis=1)).argmax()
        
        #swaps rows
        U[[i,pivot]] = U[[pivot,i]]
        L[[i,pivot],:i] = L[[pivot,i],:i]
        P[[i,pivot]] = P[[pivot,i]]
        
        #updates L and eliminates entries below i in U
        L[i+1:,i] = U[i+1:,i] / U[i,i]
        U[i+1:] = U[i+1:] - L[i+1:,i][:, np.newaxis] * U[i]
        
    return L, U, P

In [22]:
A = [[1,10**20],[1,1]]
L, U, P = modLUPdecomp(A)
print(L@U - P@A)

[[0.0 0.0]
 [0.0 0.0]]


To investigate the second problem assume without loss of generality that no permutations are needed. Let $\tilde{L}$ and $\tilde{U}$ be the calculated decomposition of $A$ with an error matrix $H$ such that $\tilde{L}\tilde{U} = A + E.$ Then $|E| \lesssim 3(n-1)\textrm{eps}(|A|+|\tilde{L}||\tilde{U}|),$ where $\lesssim$ ignores $O(\textrm{eps}^2)$ terms. Because of the example above the $\infty$-norm of $\tilde{U}$ cannot be controlled and therefore even the equilibrated LU decomposition with column pivoting is not stable.

To further analyze this behaviour we define the _growth factor_ of the LU decomposition as $\rho(A):=\dfrac{max_{i,j}|\tilde{U}|}{max_{i,j}|A|}.$ If $\rho(A)$ of order 1 then the algorithm was stable but if it is large then one can expect instability as in the example above, where $\rho(A)=2^{n-1}.$

Altough there exist cases where the growth factor grows exponentially with the dimension, in practice column pivoting is very stable. Factors $U$ that are much bigger than $A$ are seldom encountered in its applications. The reason for this phenomenon is still an open problem.

If $\tilde{x}$ is the solution of $Lx=b$ of a lower triangle matrix $L$ using forward substitution then $\tilde{x}$ solves a nearby system $(L+E)\tilde{x}=b$ with $|E| \lesssim n\, \textrm{eps}\, |L|.$ Analogously for an upper triangle matrix.

Finally the solution $\tilde{x}$ of the Gauß algorithm with column or full pivoting solves the nearby system $(A+E)\tilde{x}=b$ with $|E| \lesssim n\, \textrm{eps}\, (3|A|+5|\tilde{L}||\tilde{U}|).$ Again because of the term $|\tilde{L}||\tilde{U}|$ the Gauß algorithm is actually unstable.

## Special Systems
### Cholesky Decomposition
A matrix $A$ is called _Hermitian_ if $A^*=A$ and _positive definite_ if $x^* A x > 0 \ \ \forall x\in\mathbb{K}^n\setminus\{0\}$

Let $A$ be Hermitian and positive definite. Then there exists the unique decomposition $A = LL^*$ for a lower triangular matrix $L$ with $L_ii=0.$ If $A$ is real then $L$ is real. This is called the Cholesky decomposition.

In [23]:
def CholeskyDecomp(A):
    L = np.array(A)
    n = L.shape[0]
    
    for k in range(n):
        for j in range(k+1,n):
            L[j:,j] = L[j:,j] - L[j,k] / L[k,k] * L[j:,k]
        L[k:,k] = L[k:,k] / np.sqrt(L[k,k])
        
    return L

main complexity in inner loop: 2(n-j) flops => $\sum_{k=1}^n \sum_{j=k+1}^n 2(n-j) \approx 2 \sum_{k=1}^n \sum_{j=1}^k j \approx \sum_{k=1}^n k^2 \approx n^3$ flops

stable, since $||L||_2 = \sqrt{||A||_2}$ and in other p-norms difference is no more than $\sqrt{n}$

For $\textrm{eps}$ small enough the above algorithm guarantees a solution satisfying $\tilde{L}\tilde{L}^* = A + E$ such that $\frac{||E||}{||A||} = O(\textrm{eps})$

Furthermore $\frac{||\tilde{L}-L||}{||L||} = O(\kappa(A)\textrm{eps})$

Using the Cholesky decomposition linear systems of equations can be solved stably. Let $\tilde{x}$ be the calculated solution of $Ax=b$ for $A$ Hermitian and positive definite. Then $A+\Delta A)\tilde{x}=b$ for a $\Delta A\in\mathbb{K}^{n\times n}$ such that $\frac{||\Delta A||}{||A||} = O(\textrm{eps})$

#### LDL Decomposition
$A = LDL^*$
leads to a standard LU decomposition

#### Modified Cholesky Decomposition
$A = LL^* - D$

In [24]:
def modCholeskyDecomp(A, eps=1e-8):
    L = np.array(A)
    n = L.shape[0]
    D = np.zeros(n,n)
    
    for k in range(n):
        if L[k,k] <= 0: #matrix is not positive definite
            D[k,k] = -L[k,k] + eps
            L[k,k] = eps
        for j in range(k+1,n):
            L[j:,j] = L[j:,j] - L[j,k] / L[k,k] * L[j:,k]
        L[k:,k] = L[k:,k] / np.sqrt(L[k,k])
    
    return L, D

### Tridiagonal Matrices
A matrix $T\in\mathbb{K}^{n\times n}$ is a tridiagonal matrix if $T_{ij}=0$ for $|i-j|>1$ and all such matrices form an algebra $\textrm{Trid}(n).$

For a tridiagonal matrix $T$ the approximate solution $\tilde{x}$ of $Tx=b$ using column pivoting also solves $(T+E)\tilde{x} = b$ with $|E| \le 13\, n\, \textrm{eps}\, |T|.$

Now only $9(n-1)$ flops are required for solving a tridiagonal system of equations. The complexity is therefore linear in the dimension of the matrix.

$L$ of the Cholesky decomposition just has the lower subdiagonal and its calculation is also $O(n).$

### Band Matrices
A matrix $B\in\mathbb{K}^{n\times n}$ is called a band matrix if $B_{ij} = 0$ for all $|i-j| \ge K$ where $K\ge0$ is the bandwidth. Sometimes upper and lower bandwidths are differentiated.

If $A$ is not singular with upper and lower bandwidth $p$ and $q$ and $L$ and $U$ are computed with column pivoting then $U$ has upper bandwidth $p+q$ and every column of $L$ has at most $p+1$ non-zero elements.

$L$ of the Cholesky decomposition has the same bandwidth $K$ as $A$ and the method has complexity $O(n K^2).$

### Sparse Matrices
A matrix is called _sparse_ if a majority of entries are zeros. These matrices can be stored much more efficiently and special algorithms exist for matrix multiplication, eigenvalues, etc. that ignore all operations involving a zero.

Most of the time column pivoting is not feasible for sparse matrices since the space requirements for $L, U$ and $P$ are higher than for sparse matrices. If the matrices are big enough then minimizing space requirements is more important than the stability of the decomposition.

For example let

$$A=\begin{pmatrix}
1 & * & * & \dots & * \\
* & 2 &  &  &  \\
* &  & 3 &  &  \\
\vdots &  &  & \ddots &  \\
* &  &  &  & n \\
\end{pmatrix}.$$

Then the lower triangular matrix $L$ for $A$ of the LU decomposition or of the Cholesky decomposition in the symmetric case is full.
If a particular permutation is chosen however, namely

$$PAP^\top = \begin{pmatrix}
n &  &  &  & * \\
 & \ddots &  &  & \vdots \\
 &  & 3 &  & * \\
 &  &  & 2 & * \\
* & \dots & * & * & 1 \\
\end{pmatrix},$$

then $L$ is sparse and requires the same amount of space as $A:$

$$L = \begin{pmatrix}
* &  &  &  &  \\
 & * &  &  &  \\
 &  & * &  &  \\
 &  &  & \ddots &  \\
* & \dots & * & * & * \\
\end{pmatrix}$$

The elements in $L$ that are non-zero in $A$ but zero in $L$ are called _fill-in_. Minimizing this fill-in is done via graph theory. Most of the time however linear systems of equations with sparse matrices are solved iteratively instead of directly using a decomposition.